<a href="https://colab.research.google.com/github/cowardFm/torch_notes/blob/main/sandbox/dataloader_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データローダーを自分で書こう！


まずは画像を扱う人用のデータローダーです．

読み取る画像のpathを持ったリスト，正解のラベルがあるとします．
例えばこんな感じ

```python
data = [
    "./img1.png",
    "./img2.png",
    ...
]

label = [1, 0, ...]
```
こんな時にデータローダーを書きたいと思った場合，（コード作者の知る限り）まずはdatasetクラスを書く必要があります．

実際に書くと下のような感じ．

In [ ]:
import torch

class mydatasets(torch.utils.data.Dataset):
    def __init__(self, data, labels, transforms=None):
        self.data = data                        # データはdataという変数に格納されます．
        self.label = labels                     # ラベルはlabelという変数に．
        self.datanum = len(self.data)           # コレを設定すると，総データ数をdataloaderが管理してくれます．
        self.transforms = transforms            # data augmentation(データ拡張)の設定を保存．
        
    def __len__(self):
        return self.datanum
    
    def __getitem__(self, idx):  # 読み出し用の関数です．
        out_data = Image.fromarray(self.data[idx]) # データ読み取り
        out_label = self.label[idx]
        if self.transforms:
            out_data = self.transforms(out_data)
        return out_data, out_label

データは変数dataに保存していますが，別の名前でも構いません．
self.datanumで返す値さえ間違えなければなんでもいいです．

そのあと，datasetをインスタンス化して，Dataloaderクラスでパッケージすればよしなに使えます．
```python
dataset = mydatasets(data=data, label=label)
dataloader = torch.utils.data.DataLoader
```
こんな感じです．

# データがarrayとして保存されてる場合
(工事中)

上の読み出し用関数
```pytnon
def __getitem__()
```
を書き換えれば大丈夫です．

In [ ]:
import torch

class mydatasets(torch.utils.data.Dataset):
    def __init__(self, data, labels, transforms=None):
        self.data = data                        # データはdataという変数に格納されます．
        self.label = labels                     # ラベルはlabelという変数に．
        self.datanum = len(self.data)           # コレを設定すると，総データ数をdataloaderが管理してくれます．
        self.transforms = transforms            # data augmentation(データ拡張)の設定を保存．
        
    def __len__(self):
        return self.datanum
    
    def __getitem__(self, idx):  # 読み出し用の関数です．
        out_data = self.data[idx] # データ読み取り
        out_label = self.label[idx]
        if self.transforms:
            out_data = self.transforms(out_data)
        return out_data, out_label

ただし，arrayの形式には注意しましょう．
torch.tensorでない場合はエラーが起きた筈です．
これは出てきた要素をtorch.tensorである前提で，Dataloaderクラスがtorch.stackやらなんやらでバッチ化しようとするためです．

余談ですが，dataにcudaメモリ上のデータを渡すこともできます．
getitem内でもcudaメモリへの移行ができます．
その場合，データ拡張を含めた前処理をGPUで済ませる事ができます．非常に高速です．